## Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.metrics import f1_score

# 모델 학습을 위해 CUDA 환경 설정. : 지피유 설정
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

## 하이퍼파라미터

In [3]:
EPOCHS = 400
LR = 1e-2
BS = 100 # 2의 제곱승꼴
SEED = 41

## 시드고정

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED) # Seed 고정

## 데이터로드

In [5]:
train = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/train.csv')
train = train.drop(columns=['ID'])
val = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/val.csv')
val = val.drop(columns=['ID'])
test = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/test.csv')
test = test.drop(columns=['ID'])

print(train.shape, val.shape, test.shape)

(113842, 30) (28462, 31) (142503, 30)


## 데이터셋 생성

In [7]:
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            x = torch.from_numpy(self.df[index]).type(torch.FloatTensor)
            y = torch.FloatTensor([self.labels[index]])
            return x, y
            #self.x = self.df[index]
            #self.y = self.labels[index]
            #return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [8]:
train_dataset = MyDataset(df=train, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=6)

val_dataset = MyDataset(df = val, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=6)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
train_loader

In [ ]:
train_dataset

## 1D AutoEncoder

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.Encoder = nn.Sequential(
            nn.Linear(30,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU()
            )
        self.Decoder = nn.Sequential(
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,30)
        )
        
    def forward(self, x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x

In [9]:
# 계층 정규화
class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-5):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

        self.init_weights()

    def init_weights(self):
        self.weight.data.fill_(1.0)
        self.bias.data.zero_()

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon) # 계층정규화 완료
        return self.weight * x + self.bias # wx+b
        
# 활성화 함수
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
        
class AutoEncoder1(nn.Module):
    def __init__(self):
        super(AutoEncoder1, self).__init__()
        
        self.ln = LayerNorm(1024)
        self.ln1 = LayerNorm(512)
        self.ln2 = LayerNorm(256)
        self.ln3 = LayerNorm(128)
        
        self.upblock1 = nn.Sequential(nn.Linear(30, 128),GELU(), nn.BatchNorm1d(128))
        self.upblock2 = nn.Sequential(nn.Linear(128, 256),GELU(), nn.BatchNorm1d(256))
        self.upblock3 = nn.Sequential(nn.Linear(256, 512), GELU(), nn.BatchNorm1d(512))
        self.upblock4 = nn.Sequential(nn.Linear(512, 1024),GELU(), nn.BatchNorm1d(1024))

        self.downblock1 = nn.Sequential(nn.Linear(1024, 512),GELU(),nn.BatchNorm1d(512))
        self.downblock2 = nn.Sequential(nn.Linear(512, 256),GELU(),nn.BatchNorm1d(256))
        self.downblock3 = nn.Sequential(nn.Linear(256, 128),GELU(),nn.BatchNorm1d(128))
        self.downblock4 = nn.Sequential(nn.Linear(128, 64),GELU(),nn.BatchNorm1d(64))
        
        self.fclayer = nn.Sequential(nn.Linear(64,30))
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        upblock1_out = self.upblock1(x) # (226, 2000)->2000 # 모델Linear -> 활성화GELU -> 배치정규화BatchNorm1d 
        upblock2_out = self.upblock2(upblock1_out)# (2000,4000)->4000 # 모델Linear -> 활성화GELU -> 배치정규화BatchNorm1d 
        upblock3_out = self.upblock3(upblock2_out)# (4000,7000)->7000 # 모델Linear -> 활성화GELU -> 배치정규화BatchNorm1d 
        upblock4_out = self.upblock4(upblock3_out)# (7000,10000)->10000 # 모델Linear -> 활성화GELU -> 배치정규화BatchNorm1d 
        
        downblock1_out = self.downblock1(self.ln(upblock4_out)) #(10000, 7000)->7000 # 계층정규화LayerNorm -> 모델Linear -> 활성화GELU -> 배치정규화BatchNorm1d 
        skipblock1 = downblock1_out + upblock3_out # 7000 + 70000
        downblock2_out = self.downblock2(self.ln1(skipblock1)) #(7000, 4000)->4000 # 계층정규화LayerNorm -> 모델Linear -> 활성화GELU -> 배치정규화BatchNorm1d
        skipblock2 = downblock2_out + upblock2_out # 4000 + 4000
        downblock3_out = self.downblock3(self.ln2(skipblock2))#(4000, 2000)->2000 # 계층정규화LayerNorm -> 모델Linear -> 활성화GELU -> 배치정규화BatchNorm1d
        skipblock3 = downblock3_out + upblock1_out # 2000 + 2000
        downblock4_out = self.downblock4(self.ln3(skipblock3))#(2000, 300)->300 # 계층정규화LayerNorm -> 모델Linear -> 활성화GELU -> 배치정규화BatchNorm1d
        
        x = self.fclayer(downblock4_out) # (300, 4)->4
         
        return x # 4


## Train (학습)

In [11]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        # Loss Function
        self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self):
        self.model.to(self.device)
        best_score = 0
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader):
                x = x.float().to(self.device)
                self.optimizer.zero_grad()

                _x = self.model(x)
                loss = self.criterion(x, _x)

                loss.backward()
                self.optimizer.step()

                train_loss.append(loss.item())

            score = self.validation(self.model, 0.95)
            print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

            if self.scheduler is not None:
                self.scheduler.step(score)

            if best_score < score:
                best_score = score
                torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
    
    def validation(self, eval_model, thr):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred = []
        true = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)

                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
                pred += batch_pred
                true += y.tolist()

        return f1_score(true, pred, average='macro')

## 모델 학습

In [14]:
model = nn.DataParallel(AutoEncoder1())
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch : [0] Train loss : [0.5469581825392587] Val Score : [0.001158541795224466])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch : [1] Train loss : [0.3163006774016789] Val Score : [0.06993797991175385])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

Epoch : [397] Train loss : [0.02147681186241763] Val Score : [0.9097393418694286])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch : [398] Train loss : [0.02188367902168206] Val Score : [0.9097393418694286])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch : [399] Train loss : [0.022832832964403287] Val Score : [0.9097393418694286])


## 추론

In [16]:
model = AutoEncoder1()
model.load_state_dict(torch.load('./best_model.pth'))
model = nn.DataParallel(model)
model.eval()

DataParallel(
  (module): AutoEncoder1(
    (ln): LayerNorm()
    (ln1): LayerNorm()
    (ln2): LayerNorm()
    (ln3): LayerNorm()
    (upblock1): Sequential(
      (0): Linear(in_features=30, out_features=128, bias=True)
      (1): GELU()
      (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (upblock2): Sequential(
      (0): Linear(in_features=128, out_features=256, bias=True)
      (1): GELU()
      (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (upblock3): Sequential(
      (0): Linear(in_features=256, out_features=512, bias=True)
      (1): GELU()
      (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (upblock4): Sequential(
      (0): Linear(in_features=512, out_features=1024, bias=True)
      (1): GELU()
      (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (downblock1): Sequential(
      

In [17]:
test = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/test.csv')
test = test.drop(columns=['ID'])

In [18]:
test_dataset = MyDataset(test, False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=6)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [19]:
def prediction(model, thr, test_loader, device):
    model.to(device)
    model.eval()
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pred = []
    with torch.no_grad():
        for x in iter(test_loader):
            x = x.float().to(device)
            _x = model(x)
            
            diff = cos(x, _x).cpu().tolist()
            batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
            pred += batch_pred
    return pred

In [20]:
preds = prediction(model, 0.95, test_loader, device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [21]:
submit = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/sample_submission.csv')
submit['Class'] = preds
submit.to_csv('./submit_autoencoder1.csv', index=False)